In [ ]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd /content/drive/MyDrive/table_text_summarization_package/

/content/drive/MyDrive/table_text_summarization_package


# **Table and Text summarization**

The objective of the solution notebook is to demonstrate how to use summarization module for Finetuning and Inferencing table/text-to-insight objective.

The trainer/inference module is based on pytorch framework and can leverage GPU accelerated machine for training/inferencing.

We will see how to use the Trainer API to train a model on it. We will also demonstrate how to inference a dataset incase you already have a trained model.

## **Imports and Initializations**

In [ ]:
!pip install -r requirements.txt
!pip install --upgrade accelerate

In [ ]:
from table_text_summarization import Summarizer

# **TEXT**

#Text data preprocessing
Downloading cnn_dailymail data and data preprocessing



In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset('cnn_dailymail', '3.0.0')

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

#### saving train,val and test data in csv format

In [ ]:
# https://huggingface.co/datasets/cnn_dailymail/

In [ ]:
import pandas as pd
import json

In [ ]:
%%time
train_df =pd.DataFrame(raw_datasets['train'])
train_df.rename(columns = {'article':'text','highlights':'summary'},inplace = True)
train_df = train_df.reset_index().drop(columns=['index'])
train_df.to_csv("usecase_data/train.csv",index=False)


CPU times: user 22.9 s, sys: 2.07 s, total: 24.9 s
Wall time: 28.4 s


In [ ]:
%%time
val_df =pd.DataFrame(raw_datasets['validation'])
val_df.rename(columns = {'article':'text','highlights':'summary'},inplace = True)
val_df = val_df.reset_index().drop(columns=['index'])
val_df.to_csv("usecase_data/validation.csv",index=False)

CPU times: user 1.59 s, sys: 39.6 ms, total: 1.63 s
Wall time: 1.8 s


In [ ]:
%%time
test_df =pd.DataFrame(raw_datasets['test'])
test_df.rename(columns = {'article':'text','highlights':'summary'}, inplace = True)
test_df = test_df.reset_index().drop(columns=['index'])
test_df.to_csv("usecase_data/test.csv",index=False)

CPU times: user 806 ms, sys: 42.7 ms, total: 849 ms
Wall time: 925 ms


If summary_type is not initialized,default it consider it as "text"

In [ ]:
# Initializing Summarizer object for text-to-insight task
# summary_type = "text" or just initializing the class triggers pre processing functions required for processing text data.
model = Summarizer(summary_type="text")
# or
# model = Summarizer()

## **Fine-tuning the model**

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class internally. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

We can use pretrained-model from huggingface and it supports Seq2Seq architectures. e.g. T5, BART, GPT etc.

Below mentioned are some of the important points :

* The user should provide the training and validation file path(optional) in **json/csv format**. The text column name should be **"text"** and summary column name should be **"summary"**

* The user should also provide the output path where all the model results will be saved.

* If the model name is not specified, default model(**facebook/bart-large-cnn**) is taken into consideration

* If the model is t5  specifiy **model type =t5**, so that t5 params will get considered. else **model type = others**

* Default model params are considered for the architecture specified. We can pass [**kwargs](https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html#trainingarguments) to the train function to alter the default model params.

* The user should provide **train_prediction = True** for train data prediction and **val_prediction = True** for validation data prediction.

* **Train/validation prediction** and **eval metrics** are stored in **prediction folder** inside output folder path.
```
|--output_path/
|    |--prediction folder/
|       |--train folder
|          |--train_data_benchmarks.csv
|          |--train_generated_predictions.jsonl
|       |--validation folder/
|          |--validation_data_benchmarks.csv
|          |--validation_generated_predictions.jsonl
```


For more information the user can refer [HuggingFace Summarization](https://github.com/huggingface/transformers/tree/main/examples/pytorch/summarization)

**NOTE**: When **output_path='text'** then fine-tuned model is saved in **'text/training'** folder

In [ ]:
# model.train() function is used for fine tunning
# `train_data_path` :
# `output_path` :
# `model_name` :
# `model_type` :
# `train_prediction` :
# `val_prediction` :
# `valn_path` :
# **kwargs : Additional arguments for the models

# Using trainer function with necessary params
model.train(train_data_path="usecase_data/train_500.csv",output_path="text", model_name="t5-small",model_type='t5',valn_path="usecase_data/validation_500.csv",train_prediction=True,val_prediction=True)

# Using trainer function with additional params
# model.train(train_data_path="usecase_data/train_500.csv",output_path="text", model_name="t5-small",model_type='t5',train_prediction=True,val_prediction=True,valn_path="usecase_data/validation_500.csv",per_device_train_batch_size=2,learning_rate=5e-5,max_train_samples=50)

Executed
Model Name: t5-small
validation
training started
Trainer output:  b"05/10/2023 06:43:38 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False\n05/10/2023 06:43:38 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(\n_n_gpu=1,\nadafactor=False,\nadam_beta1=0.9,\nadam_beta2=0.999,\nadam_epsilon=1e-08,\nauto_find_batch_size=False,\nbf16=False,\nbf16_full_eval=False,\ndata_seed=None,\ndataloader_drop_last=False,\ndataloader_num_workers=0,\ndataloader_pin_memory=True,\nddp_backend=None,\nddp_bucket_cap_mb=None,\nddp_find_unused_parameters=None,\nddp_timeout=1800,\ndebug=[],\ndeepspeed=None,\ndisable_tqdm=False,\ndo_eval=True,\ndo_predict=False,\ndo_train=True,\neval_accumulation_steps=None,\neval_delay=0,\neval_steps=None,\nevaluation_strategy=no,\nfp16=False,\nfp16_backend=auto,\nfp16_full_eval=False,\nfp16_opt_level=O1,\nfsdp=[],\nfsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_g

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Evaluation output:  b"05/10/2023 06:48:15 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False\n05/10/2023 06:48:15 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(\n_n_gpu=1,\nadafactor=False,\nadam_beta1=0.9,\nadam_beta2=0.999,\nadam_epsilon=1e-08,\nauto_find_batch_size=False,\nbf16=False,\nbf16_full_eval=False,\ndata_seed=None,\ndataloader_drop_last=False,\ndataloader_num_workers=0,\ndataloader_pin_memory=True,\nddp_backend=None,\nddp_bucket_cap_mb=None,\nddp_find_unused_parameters=None,\nddp_timeout=1800,\ndebug=[],\ndeepspeed=None,\ndisable_tqdm=False,\ndo_eval=False,\ndo_predict=True,\ndo_train=False,\neval_accumulation_steps=None,\neval_delay=0,\neval_steps=None,\nevaluation_strategy=no,\nfp16=False,\nfp16_backend=auto,\nfp16_full_eval=False,\nfp16_opt_level=O1,\nfsdp=[],\nfsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},\nfsdp_min_num_params=0,\nfsdp_trans

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Evaluation output:  b"05/10/2023 06:54:23 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False\n05/10/2023 06:54:23 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(\n_n_gpu=1,\nadafactor=False,\nadam_beta1=0.9,\nadam_beta2=0.999,\nadam_epsilon=1e-08,\nauto_find_batch_size=False,\nbf16=False,\nbf16_full_eval=False,\ndata_seed=None,\ndataloader_drop_last=False,\ndataloader_num_workers=0,\ndataloader_pin_memory=True,\nddp_backend=None,\nddp_bucket_cap_mb=None,\nddp_find_unused_parameters=None,\nddp_timeout=1800,\ndebug=[],\ndeepspeed=None,\ndisable_tqdm=False,\ndo_eval=False,\ndo_predict=True,\ndo_train=False,\neval_accumulation_steps=None,\neval_delay=0,\neval_steps=None,\nevaluation_strategy=no,\nfp16=False,\nfp16_backend=auto,\nfp16_full_eval=False,\nfp16_opt_level=O1,\nfsdp=[],\nfsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},\nfsdp_min_num_params=0,\nfsdp_trans

## **Inferencing the model**

### **Using test data**

See the up-to-date list of [available models](https://huggingface.co/models?pipeline_tag=summarization)

Below mentioned are some of the important points :

* The user should provide the test path in  **json/csv format**. The text column name should be **"text"**

* The user should also provide the output path where all the model results will be saved.

* We can pass pretrained / fine tuned model path. If the model name is not specified, default model(**facebook/bart-large-cnn**) is taken into consideration

* If the model is t5  specifiy **model type =t5**, so that t5 params will get considered. else **model type = others**

We can pass [**kwargs](https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html#trainingarguments) to the predict function to alter the default model params.

(i.e) test_data_benchmarks.csv(optional -it generates only if test data has actual summary)

* **Train/validation/test prediction** and **eval metrics** are stored in **prediction folder** inside output folder path.
```
|--output_path/
|    |--prediction folder/
|       |-- train folder/
|          |--train_data_benchmarks.csv
|          |--train_generated_predictions.jsonl
|       |--validation folder/
|          |--validation_data_benchmarks.csv
|          |--validation_generated_predictions.jsonl
|       |--test folder/
|          |--test_data_benchmarks.csv
|          |--test_generated_predictions.jsonl
```

### **Using pretrained model**

In [ ]:
# model.predict() function is used for prediction
# `test_path` :
# `output_path` :
# `model_name` :
# `model_type` :
# **kwargs : Additional arguments for the models

#initializing the object for inference using pretrained model so that params get refreshed. params like learning rate etc modified for finetuning is not used.
pretrain_model = Summarizer(summary_type="text")

# Using predict function with necessary params
pretrain_model.predict(test_path="usecase_data/test.csv",output_path="text/prediction/test",model_name="t5-small",model_type="t5")

# Using predict function with additional params and finetuned model
# pretrain_model.predict(test_path="text_data/processed/test_data_5.csv",output_path="text/prediction/test",model_name="t5-small",model_type="t5",per_device_train_batch_size=2,learning_rate=5e-5,max_train_samples=50)

Evaluation output:  b"05/05/2023 06:10:04 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0distributed training: True, 16-bits training: False\n05/05/2023 06:10:04 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(\n_n_gpu=0,\nadafactor=False,\nadam_beta1=0.9,\nadam_beta2=0.999,\nadam_epsilon=1e-08,\nauto_find_batch_size=False,\nbf16=False,\nbf16_full_eval=False,\ndata_seed=None,\ndataloader_drop_last=False,\ndataloader_num_workers=0,\ndataloader_pin_memory=True,\nddp_backend=None,\nddp_bucket_cap_mb=None,\nddp_find_unused_parameters=None,\nddp_timeout=1800,\ndebug=[],\ndeepspeed=None,\ndisable_tqdm=False,\ndo_eval=False,\ndo_predict=True,\ndo_train=False,\neval_accumulation_steps=None,\neval_delay=0,\neval_steps=None,\nevaluation_strategy=no,\nfp16=False,\nfp16_backend=auto,\nfp16_full_eval=False,\nfp16_opt_level=O1,\nfsdp=[],\nfsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},\nfsdp_min_num_params=0,\nfsdp_transfor

### **Using finetuned model**

In [ ]:
# model.predict() function is used for prediction
# `test_path` :
# `output_path` :
# `model_name` :
# `model_type` :
# **kwargs : Additional arguments for the models

# Using predict function with necessary params
model.predict(test_path="usecase_data/test.csv",output_path="text/prediction/test",model_name="text/training",model_type="t5")

# Using predict function with additional params and finetuned model
# model.predict(test_path="text_data/processed/test_data_5.csv",output_path="text/prediction/test",model_name="text/training",model_type="t5",per_device_train_batch_size=2,learning_rate=5e-5,max_train_samples=50)

### **Using single context**

We need to pass context as input and it extracts the summary for the given context.

See the up-to-date list of [available models](https://huggingface.co/models?pipeline_tag=summarization)

Below mentioned are some of the important points :

* The user should provide the context as a string format.

* We can pass pretrained / fine tuned model path. If the model name is not specified, default model(**facebook/bart-large-cnn**) is taken into consideration

* If the model is t5  specifiy **model type =t5**, so that t5 params will get considered. else **model type = others**

We can pass [**kwargs](https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html#trainingarguments) to the predict function to alter the default model params.

In [ ]:
context = "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building,"\
"and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side."\
"During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure"\
"in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930."\
"It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top"\
"of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, "\
"the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

### **Using pretrained model**

In [ ]:
# model.predict() function is used for prediction
# `context` :
# `model_name` :
# **kwargs : Additional arguments for the models

#initializing the object for inference using pretrained model so that params get refreshed. params like learning rate etc modified for finetuning is not used.
pretrain_model = Summarizer(summary_type="text")

# Using predict function with necessary params
result=pretrain_model.predict(context=context,model_name="t5-small",model_type='t5')

# Using predict function with additional params and finetuned model 
# result=pretrain_model.predict(context=context,model_name="t5",model_type='t5',min_length=20)

Your max_length is set to 200, but your input_length is only 189. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=94)


### **Using finetuned model**

In [ ]:
# model.predict() function is used for prediction
# `context` :
# `model_name` :
# **kwargs : Additional arguments for the models

# Using predict function with necessary params
result=pretrain_model.predict(context=context,model_name="text/training",model_type='t5')

# Using predict function with additional params and finetuned model 
# result=pretrain_model.predict(context=context,model_name="text/training",model_type='t5',min_length=20)

## **Evaluating Model**

We can use the classical generative text evaluation metric like **BLEU**, **ROUGE** and **Semantic Similarity** scores to benchmark the model.

In [ ]:
# specify prediction data path
path="text/prediction/test/"
# reading predicted file
pred_df = open(path+'test_generated_predictions.jsonl')
pred_df = json.load(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.head(2)

,text,actual_summary,predicted_summary,model_name,summary_type,data_type
0,<table> <cell> CHIPS_ DC SUMMIT IL <header> de...,validation data table summary,cell> CHIPS_ DC SUMMIT IL header> destination ...,t5-small,table,train


In [ ]:
# get_evaluation_metrics() function is used to generate benchmarking scores on different metrics
# `actuals` : Actual text or reference
# `predicted` : Generated text or predictions

bleu,rouge_one,rouge_l,semantic,scores_df  =  get_evaluation_metrics(actuals=pred_df['actual_summary'].tolist(), predicted=pred_df['predicted_summary'].tolist())
scores_df.to_csv(os.path.join(path,"metric_scores.csv"))
print(bleu,rouge_one,rouge_l,semantic)

# **TABLE**

*   **Table-to-Insights** : The model has to generate insight/summary of a table which is converted into a free flowing text and passed as input sequence.

We will see how to easily load and preprocess the dataset for the task, and how to use the Trainer API to train a model on it. We will also demonstrate how to inference a dataset incase you already have a trained model.

The trainer/inference module is based on pytorch framework and can leverage GPU accelerated machine for training/inferencing.

## Initialize the class

In [ ]:
# Initializing Summarizer object for table-to-insight task
# summary_type = "table" triggers pre processing functions required for processing tabular data.
model = Summarizer(summary_type="table")

## **Data Preprocessing using lattice**

The preprocessing step involves conversion of tables/dataframes present in csv or excels into free flowing text and is saved as JSONs for model training/inferencing.

 - **Example folder structure and file type of raw data**
```
|-- train_folder/
|   |-- table_1.csv
|   |-- table_1.jsonl
|   |-- table_2.csv
|   |-- table_2.jsonl
|    ...
|    ...
|   |-- table_n.csv
|   |-- table_n.jsonl
```

  -> **.csv** - Contains the table in structured format which needs to be processed to flattened text

  -> **.jsonl** - Contains two fields which is Summary and Highlighted cells.
  e.g.
```
{summary : "actual summary of the data", highlighted_cells : [[1,2],[2,2],[3,2]]}
```
   - Summary represents the actual summary of the table which will be used for model training.
   - Highlighted cells represents the cells for which the summary is available or want to generate during inference.

   In the above example, `highlighted cells: [[1,2],[2,2],[3,2]]` specify the pre-process function to select data only in rows 1,2,3 and column 2 from the table and `summary: "actual summary of the data"` will have the actual summary of these 3 rows




In [ ]:
# model.pre_process() is used to pre process tables into flattened text
# `data` argument is the folder path of the data
# `data_type` argument specifies the type of the data for model training
model.pre_process(data='data/train_folder',data_type='train')

actual summary
[[1, 0], [1, 1], [1, 2]]
Executed Lattice...
Pre-processing done...


In [ ]:
model.pre_process(data="data/test_folder",data_type='test')

Executed Lattice...
Pre-processing done...


In [ ]:
model.pre_process(data="data/validation_folder",data_type='validation')

validation data table summary
[[1, 0], [1, 1], [1, 2], [1, 3], [1, 4]]
Executed Lattice...
Pre-processing done...


## **Fine-tuning the model**

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class internally. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

We can use pretrained-model from huggingface and it supports Seq2Seq architectures. e.g. T5, BART, GPT etc.

Below mentioned are some of the important points :

* The user should provide the training and validation file path(optional) in **json/csv format**. The text column name should be **"text"** and summary column name should be **"summary"**

* The user should also provide the output path where all the model results will be saved.

* If the model name is not specified, default model(**facebook/bart-large-cnn**) is taken into consideration

* If the model is t5  specifiy **model type =t5**, so that t5 params will get considered. else **model type = others**

* Default model params are considered for the architecture specified. We can pass [**kwargs](https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html#trainingarguments) to the train function to alter the default model params.

* The user should provide **train_prediction = True** for train data prediction and **val_prediction = True** for validation data prediction.

* **Train/validation** and **eval metrics** are stored in **prediction folder** inside output folder path.
```
|--output_path/
|    |--prediction folder/
|       |-- train folder/
|          |--train_data_benchmarks.csv
|          |--train_generated_predictions.jsonl
|       |--validation folder/
|          |--validation_data_benchmarks.csv
|          |--validation_generated_predictions.jsonl
```


For more information the user can refer [HuggingFace Summarization](https://github.com/huggingface/transformers/tree/main/examples/pytorch/summarization)

**NOTE**: When **output_path='table'** then fine-tuned model is saved in **'table/training'** folder

In [ ]:
# model.train() function is used for fine tunning
# `train_data_path` :
# `output_path` :
# `model_name` :
# `model_type` :
# `train_prediction` :
# `val_prediction` :
# `valn_path` :
# **kwargs : Additional arguments for the models

# Using trainer function with necessary params
model.train(train_data_path="data/train_folder/processed/train_data.json",output_path="table", model_name="t5-small",model_type="t5",valn_path="data/validation_folder/processed/validation_data.json")

# Using trainer function with additional params
# model.train(train_data_path="data/train_folder/processed/train_data.json",output_path="table", model_name="t5-small",model_type="t5",valn_path="data/validation_folder/processed/validation_data.json",train_prediction=True,val_prediction=True,per_device_train_batch_size=2,learning_rate=5e-5,max_train_samples=50)

Executed
Model Name: t5-small
validation
training started
Trainer output:  b"05/05/2023 07:34:10 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0distributed training: True, 16-bits training: False\n05/05/2023 07:34:10 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(\n_n_gpu=0,\nadafactor=False,\nadam_beta1=0.9,\nadam_beta2=0.999,\nadam_epsilon=1e-08,\nauto_find_batch_size=False,\nbf16=False,\nbf16_full_eval=False,\ndata_seed=None,\ndataloader_drop_last=False,\ndataloader_num_workers=0,\ndataloader_pin_memory=True,\nddp_backend=None,\nddp_bucket_cap_mb=None,\nddp_find_unused_parameters=None,\nddp_timeout=1800,\ndebug=[],\ndeepspeed=None,\ndisable_tqdm=False,\ndo_eval=True,\ndo_predict=False,\ndo_train=True,\neval_accumulation_steps=None,\neval_delay=0,\neval_steps=None,\nevaluation_strategy=no,\nfp16=False,\nfp16_backend=auto,\nfp16_full_eval=False,\nfp16_opt_level=O1,\nfsdp=[],\nfsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad

## **Inferencing the model**

### **Using test data**

See the up-to-date list of [available models](https://huggingface.co/models?pipeline_tag=summarization)

Below mentioned are some of the important points :

* The user should provide the test path in  **json/csv format**. The text column name should be **"text"**

* The user should also provide the output path where all the model results will be saved.

* We can pass pretrained / fine tuned model path. If the model name is not specified, default model(**facebook/bart-large-cnn**) is taken into consideration

* If the model is t5  specifiy **model type =t5**, so that t5 params will get considered. else **model type = others**

We can pass [**kwargs](https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html#trainingarguments) to the predict function to alter the default model params.

(i.e) test_data_benchmarks.csv(optional -it generates only if test data has actual summary)

* **Train/validation/test prediction** and **eval metrics** are stored in **prediction folder** inside output folder path.
```
|--output_path/
|    |--prediction folder/
|       |-- train folder/
|          |--train_data_benchmarks.csv
|          |--train_generated_predictions.jsonl
|       |--validation folder/
|          |--validation_data_benchmarks.csv
|          |--validation_generated_predictions.jsonl
|       |--test folder/
|          |--test_data_benchmarks.csv
|          |--test_generated_predictions.jsonl
```

### **Using pretrained model**

In [ ]:
# model.predict() function is used for prediction
# `test_path` :
# `output_path` :
# `model_name` :
# `model_type` :
# **kwargs : Additional arguments for the models

#initializing the object for inference using pretrained model so that params get refreshed. params like learning rate etc modified for finetuning is not used.
pretrain_model = Summarizer(summary_type="table")

# Using predict function with necessary params
pretrain_model.predict(test_path="data/test_folder/processed/test_data.json",output_path="table/prediction/test", model_name="t5-small",model_type="t5")

# Using predict function with additional params
# pretrain_model.predict(test_path="data/test_folder/processed/test_data.json",output_path="table/prediction/test",model_name="t5-small",model_type="t5",per_device_train_batch_size=2,learning_rate=5e-5,max_train_samples=50)


Evaluation output:  b"05/05/2023 06:10:04 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0distributed training: True, 16-bits training: False\n05/05/2023 06:10:04 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(\n_n_gpu=0,\nadafactor=False,\nadam_beta1=0.9,\nadam_beta2=0.999,\nadam_epsilon=1e-08,\nauto_find_batch_size=False,\nbf16=False,\nbf16_full_eval=False,\ndata_seed=None,\ndataloader_drop_last=False,\ndataloader_num_workers=0,\ndataloader_pin_memory=True,\nddp_backend=None,\nddp_bucket_cap_mb=None,\nddp_find_unused_parameters=None,\nddp_timeout=1800,\ndebug=[],\ndeepspeed=None,\ndisable_tqdm=False,\ndo_eval=False,\ndo_predict=True,\ndo_train=False,\neval_accumulation_steps=None,\neval_delay=0,\neval_steps=None,\nevaluation_strategy=no,\nfp16=False,\nfp16_backend=auto,\nfp16_full_eval=False,\nfp16_opt_level=O1,\nfsdp=[],\nfsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},\nfsdp_min_num_params=0,\nfsdp_transfor

### **Using Finetuned model**

In [ ]:
# model.predict() function is used for prediction
# `test_path` :
# `output_path` :
# `model_name` :
# `model_type` :
# **kwargs : Additional arguments for the models

# Using predict function with necessary params for finetuned model
model.predict(test_path="data/test_folder/processed/test_data.json",output_path="table/prediction/test", model_name="table/training",model_type="t5")

## Using predict function with additional params
# model.predict(test_path="data/test_folder/processed/test_data.json",output_path="table/prediction/test",model_name="table/training",model_type="t5",per_device_train_batch_size=2,learning_rate=5e-5,max_train_samples=50)


### **Using single context**

We need to pass context as input and it extracts the summary for the given context.

See the up-to-date list of [available models](https://huggingface.co/models?pipeline_tag=summarization)

Below mentioned are some of the important points :

* The user should provide the context as a string format.

* We can pass pretrained / fine tuned model path. If the model name is not specified, default model(**facebook/bart-large-cnn**) is taken into consideration

* If the model is t5  specifiy **model type =t5**, so that t5 params will get considered. else **model type = others**

We can pass [**kwargs](https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html#trainingarguments) to the predict function to alter the default model params.

In [ ]:
context = "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building,"\
"and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side."\
"During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure"\
"in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930."\
"It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top"\
"of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, "\
"the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

### **Using pretrained model**

In [ ]:
# model.predict() function is used for prediction
# `context` :
# `model_name` :
# **kwargs : Additional arguments for the models

#initializing the object for inference using pretrained model so that params get refreshed. params like learning rate etc modified for finetuning is not used.
pretrain_model = Summarizer(summary_type="table")

# Using predict function with necessary params
result=pretrain_model.predict(context=context,model_name="t5-small",model_type="t5")

# Using predict function with additional params
# result=pretrain_model.predict(context=context,model_name="t5-small",,model_type="t5",learning_rate=5e-5,min_length=5, max_length=20)

### **Using finetuned model**

In [ ]:
# model.predict() function is used for prediction
# `context` :
# `model_name` :
# **kwargs : Additional arguments for the models

# Using predict function with necessary params
result=model.predict(context=context,model_name="table/training",model_type="t5")

# Using predict function with additional params
# result=model.predict(context=context,model_name="table/training",,model_type="t5",learning_rate=5e-5,min_length=5, max_length=20)

## **Evaluating Model**

We can use the classical generative text evaluation metric like **BLEU**, **ROUGE** and **Semantic Similarity** scores to benchmark the model.

In [ ]:
# specify prediction data path
path="table/prediction/test/"
# reading predicted file
pred_df = open(path+'test_generated_predictions.jsonl')
pred_df = json.load(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.head(2)

,text,actual_summary,predicted_summary,model_name,summary_type,data_type
0,<table> <cell> CHIPS_ DC SUMMIT IL <header> de...,validation data table summary,cell> CHIPS_ DC SUMMIT IL header> destination ...,t5-small,table,train


In [ ]:
# get_evaluation_metrics() function is used to generate benchmarking scores on different metrics
# `actuals` : Actual text or reference
# `predicted` : Generated text or predictions

bleu,rouge_one,rouge_l,semantic,scores_df  =  get_evaluation_metrics(actuals=pred_df['actual_summary'].tolist(), predicted=pred_df['predicted_summary'].tolist())
scores_df.to_csv(os.path.join(path,"metric_scores.csv"))
print(bleu,rouge_one,rouge_l,semantic)

0.0 0.05263157894736842 0.05263157894736842 0.15557736158370972
